Testing Playground :)

In [11]:
# %pip install onnx==1.12.0

     --------------------------------------- 11.5/11.5 MB 10.7 MB/s eta 0:00:00
     -------------------------------------- 903.8/903.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: onnx
    Found existing installation: onnx 1.13.1
    Uninstalling onnx-1.13.1:
      Successfully uninstalled onnx-1.13.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.1 which is incompatible.
googleapis-common-protos 1.59.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-api-core 2.11.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [86]:
import cv2
import torch
import numpy as np
import yaml
import os

In [87]:
opencv_net = cv2.dnn.readNet('yolov5/models/yolov5s.onnx')
imput_vid = cv2.VideoCapture('input/1.mp4')
filename = 'input/1.mp4'

classes = []

with open('yolov5/data/coco.yaml', 'r') as file:
        dict = yaml.load(file, Loader=yaml.FullLoader)
        arr = list(dict['names'].items())
        for key in range(len(arr)):
                classes.append(arr[key][1]) # coco 80 classes name

if (imput_vid.isOpened() == True):
        width = int(imput_vid.get(3))
        height = int(imput_vid.get(4))
        fps = imput_vid.get(5)
        print('Weight x Height FPS: ', width, 'x' ,height, fps)

        fourcc = cv2.VideoWriter_fourcc('F', 'M', 'P', '4')
        video = cv2.VideoWriter(os.path.basename(filename).split(".")[-1] + '_output.avi', fourcc, fps, (width,height))
else:
        print('Can''t load the file, try another one :(')



Weight x Height FPS:  1920 x 1080 29.97002997002997


In [88]:
def preprocess(input, net):
        blob = cv2.dnn.blobFromImage(input, 1/255, (640, 640), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        output = net.forward(net.getUnconnectedOutLayersNames())

        return output

In [89]:
def postprocess(input, outputs):
        class_ids = []
        confidences = []
        boxes = []

        for rows in outputs:
                for detections in rows:
                        scores = detections[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.5:
                            center_x = int(detections[0] * width)
                            center_y = int(detections[1] * height)
                            w = int(detections[2] * width)
                            h = int(detections[3] * height)

                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)

                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) # (boxes, class scores, score threshold, nms threshold)
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                cv2.rectangle(input, (x, y), (x + w, y + h), (0,0,0), 3)

        return input

In [90]:
flg, frames = imput_vid.read()

In [77]:
# detections = preprocess(imput_vid, opencv_net)